# 01. Data Exploration - GreenSpace Survey

This notebook explores the survey response data and images to understand patterns, distributions, and data quality for training our multi-task CNN.

## Objectives
1. Load and inspect survey responses 
2. Analyze label distributions and correlations
3. Examine image dataset characteristics
4. Identify data quality issues
5. Create visualizations for insights


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import yaml

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Configuration
DATA_DIR = Path("../data")
CONFIG_PATH = Path("../config/model_config.yaml")

print("Libraries imported successfully!")
print(f"Data directory: {DATA_DIR}")
print(f"Config path: {CONFIG_PATH}")
